In [1]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# $matplotlib --listでoptionを表示
#%matplotlib inline
%matplotlib inline

import datetime
import os

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')
import gc

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.datasets import load_iris
iris_data = load_iris()
iris_df = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris_df["target"] = iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
X_name = list(iris_df.columns.values[:4])
Y_name = iris_df.columns.values[4]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(iris_df[X_name].values, iris_df[Y_name].values,
                                                   test_size=0.2, random_state=18461)

# スケーリング
sklearnだと  

* standard scaler  
* robust scaler  
* min-max scaler  

があるっぽい  

In [6]:
s_scaler = StandardScaler()

In [7]:
X_train = s_scaler.fit_transform(X_train)
X_test = s_scaler.transform(X_test)

# Logistic Regression

In [8]:
kfold = KFold(n_splits=5)

In [9]:
oof = np.zeros((len(X_train), 3))
predictions = np.zeros((5, len(X_test), 3))

for fold_, (trn_idx, val_idx) in enumerate(kfold.split(X_train, X_train)):
    print("Fold {}".format(fold_))
    # C=1000 → 正則化 lambda=0.0001
    lr = LogisticRegression(C=1000.0, random_state=846)
    lr.fit(X_train[trn_idx], Y_train[trn_idx])
    
    oof[val_idx] = lr.predict_proba(X_train[val_idx])
    predictions[fold_] = lr.predict_proba(X_test)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [10]:
pred = np.argmax(np.mean(predictions, axis=0), axis=1)
(Y_test == pred).sum() / len(Y_test)

0.9666666666666667

In [11]:
predictions

array([[[2.26505465e-11, 2.64325316e-01, 7.35674684e-01],
        [2.66505810e-04, 9.74603169e-01, 2.51303251e-02],
        [1.95687008e-10, 4.61140464e-01, 5.38859536e-01],
        [7.66346841e-03, 9.92334341e-01, 2.19091215e-06],
        [4.84087662e-04, 9.99400526e-01, 1.15386782e-04],
        [7.04275957e-09, 3.55306520e-01, 6.44693473e-01],
        [8.98805140e-01, 1.01194860e-01, 4.56811608e-21],
        [8.17113056e-01, 1.82886944e-01, 1.94229845e-20],
        [9.07933753e-01, 9.20662467e-02, 5.35050591e-21],
        [2.04803546e-09, 5.36145174e-02, 9.46385481e-01],
        [1.06449445e-04, 9.99630588e-01, 2.62962204e-04],
        [2.06060183e-04, 9.97735055e-01, 2.05888530e-03],
        [4.28720615e-09, 9.60216293e-02, 9.03978366e-01],
        [3.07819128e-05, 9.89983199e-01, 9.98601861e-03],
        [1.18504486e-09, 9.98649909e-02, 9.00135008e-01],
        [7.04678509e-10, 2.53344496e-01, 7.46655503e-01],
        [9.16975659e-01, 8.30243412e-02, 2.02984983e-21],
        [9.259

# SVM

In [12]:
from sklearn.svm import SVC

In [13]:
oof = np.zeros((len(X_train), 3))
predictions = np.zeros((5, len(X_test), 3))

for fold_, (trn_idx, val_idx) in enumerate(kfold.split(X_train, X_train)):
    print("Fold {}".format(fold_))
    # 線形: linear, or rbfとか
    # probability=Trueをつけると，predict_probaが使える
    model = SVC(kernel="linear", C=1.0, random_state=863, probability=True)
    model.fit(X_train[trn_idx], Y_train[trn_idx])
    
    oof[val_idx] = model.predict_proba(X_train[val_idx])
    predictions[fold_] = model.predict_proba(X_test)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [14]:
pred = np.argmax(np.mean(predictions, axis=0), axis=1)
(Y_test == pred).sum() / len(Y_test)

1.0

# Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

スケーリングしない

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(iris_df[X_name].values, iris_df[Y_name].values,
                                                   test_size=0.2, random_state=18461)

In [17]:
oof = np.zeros((len(X_train), 3))
predictions = np.zeros((5, len(X_test), 3))

for fold_, (trn_idx, val_idx) in enumerate(kfold.split(X_train, X_train)):
    print("Fold {}".format(fold_))
    model = RandomForestClassifier(criterion="entropy", n_estimators=10, random_state=1498, n_jobs=2)
    model.fit(X_train[trn_idx], Y_train[trn_idx])
    
    oof[val_idx] = model.predict_proba(X_train[val_idx])
    predictions[fold_] = model.predict_proba(X_test)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [18]:
pred = np.argmax(np.mean(predictions, axis=0), axis=1)
(Y_test == pred).sum() / len(Y_test)

0.9666666666666667

In [19]:
model.feature_importances_

array([0.08060332, 0.00937198, 0.52115189, 0.38887281])

In [20]:
indices = np.argsort(model.feature_importances_)[::-1]
for i in indices:
    print("{}: {}".format(X_name[i], model.feature_importances_[i]))

petal length (cm): 0.5211518890037162
petal width (cm): 0.38887281370159066
sepal length (cm): 0.08060332045666928
sepal width (cm): 0.00937197683802388
